# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,broome,42.2506,-75.8330,5.67,76,20,3.09,US,1699919339
1,1,lillesand,58.2488,8.3778,1.05,80,100,11.32,NO,1699919340
2,2,dudinka,69.4058,86.1778,-16.32,88,69,4.75,RU,1699919341
3,3,edinburgh of the seven seas,-37.0676,-12.3116,13.64,77,7,11.14,SH,1699919128
4,4,west island,-12.1568,96.8225,25.99,83,75,4.12,CC,1699919045


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"    
)

# Display the map plot
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [43]:
# Narrow down cities that fit criteria and drop any results with null values
filtered = city_data_df[(city_data_df["Country"] == "US") & (city_data_df["Max Temp"] >= 19)]

# Drop any rows with null values
filtered = filtered.dropna()

# Display sample data
filtered

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
107,107,hawaiian paradise park,19.5933,-154.9731,27.73,72,40,4.63,US,1699919395
113,113,holualoa,19.6228,-155.9522,27.65,74,40,4.63,US,1699919294
180,180,kapa'a,22.0752,-159.3190,29.99,81,75,8.23,US,1699919527
218,218,wailua homesteads,22.0669,-159.3780,29.24,81,75,8.23,US,1699919573
273,273,corona de tucson,31.9654,-110.7756,25.75,27,0,2.68,US,1699919450
290,290,newman,37.3138,-121.0208,24.19,46,55,0.45,US,1699919382
355,355,lompoc,34.6391,-120.4579,21.12,65,0,5.36,US,1699919730
439,439,lihue,21.9789,-159.3672,29.58,76,75,8.23,US,1699919811
455,455,hilo,19.7297,-155.0900,27.85,72,40,4.63,US,1699919753
475,475,la'ie,21.6477,-157.9253,30.98,76,86,4.47,US,1699919843


### Step 3: Create a new DataFrame called `hotel_df`.

In [44]:
# Create a copy of the original DataFrame with selected columns
hotel_df = filtered.copy()[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty "Hotel Name" column
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
107,hawaiian paradise park,US,19.5933,-154.9731,72,
113,holualoa,US,19.6228,-155.9522,74,
180,kapa'a,US,22.0752,-159.3190,81,
218,wailua homesteads,US,22.0669,-159.3780,81,
273,corona de tucson,US,31.9654,-110.7756,27,
290,newman,US,37.3138,-121.0208,46,
355,lompoc,US,34.6391,-120.4579,65,
439,lihue,US,21.9789,-159.3672,76,
455,hilo,US,19.7297,-155.0900,72,
475,la'ie,US,21.6477,-157.9253,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [45]:
# Set the parameters for the type of search
limit = 1
radius = 10000
categories = "accommodation.hotel"

# set up a parameters dictionary
params = {
    "categories":categories,
    "radius":radius,
    "limit":limit,
    "apiKey":geoapify_key    
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
hawaiian paradise park - nearest hotel: No hotel found
holualoa - nearest hotel: Kona Hotel
kapa'a - nearest hotel: Pono Kai Resort
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
corona de tucson - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
lihue - nearest hotel: Kauai Palms
hilo - nearest hotel: Dolphin Bay Hotel
la'ie - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
haiku-pauwela - nearest hotel: Inn At Mama's Fish House


,City,Country,Lat,Lng,Humidity,Hotel Name
107,hawaiian paradise park,US,19.5933,-154.9731,72,No hotel found
113,holualoa,US,19.6228,-155.9522,74,Kona Hotel
180,kapa'a,US,22.0752,-159.3190,81,Pono Kai Resort
218,wailua homesteads,US,22.0669,-159.3780,81,Hilton Garden Inn Kauai Wailua Bay
273,corona de tucson,US,31.9654,-110.7756,27,No hotel found
290,newman,US,37.3138,-121.0208,46,No hotel found
355,lompoc,US,34.6391,-120.4579,65,Embassy Suites by Hilton Lompoc Central Coast
439,lihue,US,21.9789,-159.3672,76,Kauai Palms
455,hilo,US,19.7297,-155.0900,72,Dolphin Bay Hotel
475,la'ie,US,21.6477,-157.9253,76,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [46]:
%%capture --no-display

# Configure the map
hover_cols = ["City", "Country", "Hotel Name"]

map_plot1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols=hover_cols    
)

# Display the map plot
map_plot1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)